In [1]:
import requests
from IPython.display import Image, display, clear_output
import time
from urllib.parse import urlencode
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

%matplotlib inline


In [2]:
# Function to set motors
def set_motors(jetbot_ip, left_speed, right_speed):
    params = {'left': left_speed, 'right': right_speed}
    url = f'http://{jetbot_ip}:8080/set_motors?{urlencode(params)}'
    response = requests.get(url)
    if response.status_code == 200:
        print("Motors set successfully")
    else:
        print("Failed to set motors")

In [3]:
# Function to execute stop command
def stop_robot(jetbot_ip):
    url = f'http://{jetbot_ip}:8080/stop'
    response = requests.get(url)
    if response.status_code == 200:
        print("Stop command executed successfully")
    else:
        print("Failed to execute stop command")

In [4]:

# A function that continuously streams video and allows saving images on pressing 's' and quitting on 'q'
def take_pictures_loop(jetbot_ip):
    folder = 'training_images'
    os.makedirs(folder, exist_ok=True)  # Create the folder if it doesn't exist

    url = f'http://{jetbot_ip}:8080/camera'

    while True:
        response = requests.get(url)
        img_array = np.frombuffer(response.content, dtype=np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        if image is not None:
            cv2.imshow("JetBot Camera - Press 's' to save, 'q' to quit", image)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('s'):  # Save image on pressing 's'
                # Count the number of files in the folder
                file_count = len([name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))])

                # Generate the new file name
                new_file_name = f'img{file_count + 1}.jpg'
                new_file_path = os.path.join(folder, new_file_name)

                cv2.imwrite(new_file_path, image)
                print(f"Image saved as {new_file_path}")
            elif key == ord('q'):  # Quit on pressing 'q'
                break
        else:
            print("Failed to capture image")

    cv2.destroyAllWindows()

In [5]:
# # from ultralytics import YOLO

# # Load the YOLOv8 model
# model = YOLO("yolov8n.pt")  # You can replace "yolov8n.pt" with other YOLOv8 variants like "yolov8s.pt"

# # Train the model on the dataset
# model.train(data="datasets/data.yaml", epochs=50, imgsz=224)

# # Save the trained model
# model.save("models/yolov8n_trained.pt")

In [6]:
# Function to do object detection on all images in a folder and plot results
def detect_objects_in_folder(folder_path):
    images = []
    titles = []
    model = YOLO('models/yolov8n_trained.pt')  # Load the YOLOv8 model

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            image_path = os.path.join(folder_path, filename)
            print(f"Processing {image_path}...")

            # Load the image
            image = cv2.imread(image_path)

            labels = ['robot', 'target']

            # Perform object detection
            results = model.predict(source=image, conf=0.25, save=False)

            # Annotate the image with detections and calculate angles
            bounding_boxes = []
            for result in results:
                boxes = result.boxes.xyxy  # Bounding boxes
                confidences = result.boxes.conf  # Confidence scores
                class_ids = result.boxes.cls  # Class IDs

                for box, conf, class_id in zip(boxes, confidences, class_ids):
                    x1, y1, x2, y2 = map(int, box)
                    label = f"{labels[int(class_id)]}: {conf:.2f}"   
                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Add bounding box for angle calculation
                    bounding_boxes.append((x1, y1, x2, y2))

            # Calculate angles for detected objects
            angles = calculate_object_angles(bounding_boxes, image_width=image.shape[1], image_height=image.shape[0], fov_deg=160)
            print(f"Angles for {filename}: {angles}")

            # Convert BGR to RGB for matplotlib
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image_rgb)
            titles.append(filename)

    # Plot the images with detections
    num_images = len(images)
    cols = 3
    rows = (num_images + cols - 1) // cols  # Calculate rows needed

    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()

    for i, ax in enumerate(axes):
        if i < num_images:
            ax.imshow(images[i])
            ax.set_title(titles[i])
            ax.axis('off')
        else:
            ax.axis('off')  # Hide unused subplots

    plt.tight_layout()
    plt.show()

In [7]:
def print_camera_yolo(jetbot_ip):
    # Load the trained YOLOv8 model
    model = YOLO("models/yolov8n_trained.pt")  # Load the trained model

    url = f'http://{jetbot_ip}:8080/camera'
    
    while True:
        response = requests.get(url)
        img_array = np.frombuffer(response.content, dtype=np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        if image is not None:
            # Run YOLO detection (pass BGR image directly)
            results = model.predict(image, verbose=False)

            bounding_boxes = []
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    bounding_boxes.append((x1, y1, x2, y2))

            # Calculate angles for detected objects
            angles = calculate_object_angles(bounding_boxes, image_width=image.shape[1], image_height=image.shape[0], fov_deg=160)

            for result, angle_info in zip(results, angles):
                boxes = result.boxes
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    label = model.names[cls_id]

                    # Add horizontal angle to the label
                    horizontal_angle = angle_info["horizontal_angle_deg"]
                    label_with_angle = f'{label} {conf:.2f}'
                    angle_text = f'Angle: {horizontal_angle:.2f}°'
                    cv2.putText(image, angle_text, (x1, y1 - 25),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

                    # Draw bounding box and label
                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(image, label_with_angle, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

                    # Draw center point
                    center_x = (x1 + x2) // 2
                    center_y = (y1 + y2) // 2
                    cv2.circle(image, (center_x, center_y), 5, (0, 0, 255), -1)
                    cv2.putText(image, f"({center_x},{center_y})", (x1, y2 + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Display the processed image
            cv2.imshow("JetBot Camera with YOLO Detection", image)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


In [8]:
def turn_x_degrees(jetbot_ip,degrees):
    V_MAX = 0.2  # Adjust this value as needed
    WHEEL_BASE = 0.1  # Distance between the wheels in meters
    if abs(degrees) > 0.5:
        degrees = 0.5*(degrees/abs(degrees))
    print("Degrees:", degrees)

    x = (degrees*0.1)/V_MAX
    print(x)
    if abs(degrees) > 0.05:
        if degrees > 0:
            left_speed = V_MAX
            right_speed = V_MAX * (1 - (degrees * 0.1) / WHEEL_BASE)
        else:
            left_speed = V_MAX * (1 + (degrees * 0.1) / WHEEL_BASE)
            right_speed = V_MAX
    else:
        left_speed = V_MAX
        right_speed = V_MAX

    print("Left Speed:", left_speed)
    print("Right Speed:", right_speed)
    # Set the motors
    set_motors(jetbot_ip, left_speed, right_speed)

In [9]:
def simple_pso_drive(jetbot_ip, fov_deg=160, window_name="JetBot Camera"):
    """
    A simplified PSO-like algorithm for robots to detect objects, calculate the average position,
    and drive towards the average angle, while displaying the camera stream with bounding boxes.
    """

    model = YOLO("models/yolov8n_trained.pt")  # Load the trained model

    url = f'http://{jetbot_ip}:8080/camera'


    while True:
        # Capture image from the robot's camera
        response = requests.get(url)
        img_array = np.frombuffer(response.content, dtype=np.uint8)
        image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

        if image is not None:
            # Detect objects using YOLO model
            results = model.predict(image, verbose=False)

            # Extract bounding box centers and draw bounding boxes
            centers = []
            to_close = False
            center_points = {}
            center_points[0] = []
            center_points[1] = []

            target_weight = 0.6



            for result in results:
                boxes = result.boxes
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    center_x = (x1 + x2) // 2
                    center_y = (y1 + y2) // 2
                    class_id = int(box.cls[0])                    

                    centers.append((center_x, center_y))
                    center_points[class_id].append(center_x)

                    # Draw bounding box and center point
                    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.circle(image, (center_x, center_y), 5, (0, 0, 255), -1)
                    cv2.putText(image, f"({center_x}, {center_y})", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                    
                    print(f"X2-X1: {x2-x1}, Class ID: {class_id}")
                    if (x2-x1) > 55 and class_id == 0:
                        to_close = True
                    if (x2-x1) > 60 and class_id == 1:
                        to_close = True
                    

            if centers and not to_close:
                # Calculate the average center point
                # avg_center = np.mean(centers, axis=0)
                #reweight the centerpoints
                target_point, robot_point = 0, 0
                if len(center_points[0]) == 0:
                    target_weight = 1
                if len(center_points[1]) == 0:
                    target_weight = 0
                if len(center_points[1]) > 0:
                    target_point = (np.array(center_points[1])*target_weight)[0]
                print("CENTERPOINTS", center_points)
                if len(center_points[0]) > 0:
                    robot_point = np.mean((np.array(center_points[0])*(1-target_weight))/len(center_points[0]))
                avg_center = (target_point+ robot_point)
                print("AVG_CENTER", avg_center)

                # Calculate the horizontal angle to the average center
                image_width = image.shape[1]
                x_offset = (avg_center - image_width / 2) / (image_width / 2)
                horizontal_angle = x_offset * (fov_deg / 2)

                print(f"Average Center: {avg_center}, Horizontal Angle: {horizontal_angle:.2f}°")

                # Drive the robot towards the calculated angle
                turn_x_degrees(jetbot_ip, horizontal_angle/100)
            elif to_close:
                stop_robot(jetbot_ip)
            
            else:
                print("No objects detected. Stopping robot.")
                set_motors(jetbot_ip, -0.1, 0.1)

            # Display the processed image with bounding boxes
            cv2.imshow(window_name, image)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_robot(jetbot_ip)
            break

    cv2.destroyAllWindows()

In [10]:
def calculate_object_angles(bounding_boxes, image_width, image_height, fov_deg):
    angles = []

    for box in bounding_boxes:
        x_min, y_min, x_max, y_max = box
        x_center = (x_min + x_max) / 2
        y_center = (y_min + y_max) / 2

        # Calculate horizontal angle
        x_offset = (x_center - image_width / 2) / (image_width / 2)
        horizontal_angle = x_offset * (fov_deg / 2)


        angles.append({
            "center": (x_center, y_center),
            "horizontal_angle_deg": horizontal_angle,
        })

    return angles



In [11]:
import multiprocessing

# Shared flag to stop all threads
stop_threads = multiprocessing.Event()

# Function to run simple_pso_drive for multiple robots in parallel
def run_multiple_robots(jetbot_ips, fov_deg=160):
    def robot_process(ip):
        # Create a unique window name for each robot
        window_name = f"JetBot Camera - {ip}"
        
        # Run the simple_pso_drive function with the unique window name
        simple_pso_drive(ip, fov_deg, window_name)

    processes = []

    for ip in jetbot_ips:
        process = multiprocessing.Process(target=robot_process, args=(ip,))
        processes.append(process)
        process.start()

    # Wait for all processes to complete
    for process in processes:
        process.join()

# Example usage
# jetbot_ip_list = ['194.47.156.140']
jetbot_ip_list = ['194.47.156.140', '194.47.156.221']
run_multiple_robots(jetbot_ip_list, fov_deg=160)

/home/v3/Documents/School/robotics/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/v3/Documents/School/robotics/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


X2-X1: 12, Class ID: 1
X2-X1: 10, Class ID: 0
X2-X1: 11, Class ID: 0
CENTERPOINTS {0: [138, 200], 1: [19]}
AVG_CENTERX2-X1: 15, Class ID: 0 
45.199999999999996X2-X1: 12, Class ID: 0

Average Center: 45.199999999999996, Horizontal Angle: -47.71°
CENTERPOINTSDegrees:  {0: [210, 56], 1: []}-0.4771428571428572

-0.2385714285714286AVG_CENTER
 Left Speed:66.5 0.10457142857142857

Average Center: 66.5, Horizontal Angle: -32.50°Right Speed:
 Degrees:0.2 
-0.325
-0.1625
Left Speed: 0.135
Right Speed: 0.2
Motors set successfullyMotors set successfully

X2-X1: 15, Class ID: 1
X2-X1: 10, Class ID: 0
CENTERPOINTS {0: [146], 1: [24]}
AVG_CENTER 72.80000000000001
Average Center: 72.80000000000001, Horizontal Angle: -28.00°
Degrees: -0.2799999999999999
-0.13999999999999996
Left Speed: 0.14400000000000002
Right Speed: 0.2
Motors set successfully
X2-X1: 9, Class ID: 0
CENTERPOINTS {0: [218], 1: []}
AVG_CENTER 218.0
Average Center: 218.0, Horizontal Angle: 75.71°
Degrees: 0.5
0.25
Left Speed: 0.2
Right S

In [ ]:
center_points = {}
center_points[0] = []
center_points[1] = []
center_points[0].append(40)
center_points[0].append(50)
center_points[1].append(-40)
import numpy as np
TARGET_WEIGHT = 0.5
target_point = np.array(center_points[1])*TARGET_WEIGHT
robot_point = np.mean((np.array(center_points[0])*(1-TARGET_WEIGHT))/len(center_points[0]))
angle = (target_point[0] + robot_point)

print(target_point)
print(robot_point)
print(angle)
print(center_points[0])
